In [0]:
import tensorflow as tf
from IPython.display import display
import pandas as pd
import seaborn as sns
import tensorflow.keras.backend as K
import numpy as np
import os
import gc
import glob
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import BatchNormalization, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
%matplotlib inline

In [0]:
def tf_reset_graph(model=None):
    tf.reset_default_graph()
    K.clear_session()
    gc.collect()

def tf_reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint = None
    reduce_lr = None
    early_stopping = None
    tensorboard = None

Flag to decide whether to use colab or not

Also prints out the platform processor if Colab is used

In [28]:
use_colab = True
if use_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  proj_path = 'drive/My Drive/Colab/Workspace/'
  import platform
  print("Processor: ", platform.processor())
  !nvidia-smi
else:
  proj_path=''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processor:  x86_64
Wed Sep 25 21:26:33 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    85W / 149W |   8428MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------------------

Describing the paths to the train, test and validation datasets

In [5]:
image_ext = '.jpeg'
data_dir = proj_path+'dataset/'
test_dir = data_dir+'test/'
train_dir = data_dir+'train/'
val_dir = data_dir+'val/'
normals = 'NORMAL/'
infected = 'PNEUMONIA/'
training_normal_nos = len(glob.glob1(train_dir+normals,'*'+image_ext))
training_infected_nos = len(glob.glob1(train_dir+infected,'*'+image_ext))
training_nos = training_infected_nos+training_normal_nos
testing_normal_nos = len(glob.glob1(test_dir+normals,'*'+image_ext))
testing_infected_nos = len(glob.glob1(test_dir+infected,'*'+image_ext))
testing_nos = testing_infected_nos+testing_normal_nos
validation_normal_nos = len(glob.glob1(val_dir+normals,'*'+image_ext))
validation_infected_nos = len(glob.glob1(val_dir+infected,'*'+image_ext))
validation_nos = validation_infected_nos+validation_normal_nos

print('Total number of training images: ', training_nos)
print('Total number of testing images: ', testing_nos)
print('Total number of validation images: ', validation_nos)

Total number of training images:  5230
Total number of testing images:  624
Total number of validation images:  16


A custom function to create ImageDataGenerator while using images from the 'path'

In [0]:
def create_image_data_generator(target_size, rescale, path, batch_size, class_mode = 'categorical', shuffle=True, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True):

    data_generator = ImageDataGenerator(rescale=rescale,
                                        shear_range=shear_range,
                                        zoom_range=zoom_range,
                                        horizontal_flip=horizontal_flip)

    return data_generator.flow_from_directory(path,
                                       class_mode=class_mode,
                                       target_size=target_size,
                                       shuffle=shuffle,
                                       batch_size=batch_size)

rescaling and defining target size, class mode is kept categorical, although one can also use binary since it's a binary classification problem.

test, train and validation image data generators are created

In [7]:
rescale_factor = 1./255
target_size = (150,150)
class_mode = 'categorical'
training_batch_size = 163
train_data_generator = create_image_data_generator(target_size=target_size, 
                                                   rescale=rescale_factor, 
                                                   path=train_dir, 
                                                   batch_size=training_batch_size)

test_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=test_dir,
                                                 batch_size=testing_nos)

val_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=val_dir,
                                                 batch_size=validation_nos)

Found 5230 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


Callbacks are described below. earlystopping has been removed since the model stopped too early due to bad val_loss

In [0]:
tf_reset_callbacks()
tf_reset_graph()
base = proj_path+'bin'
model_dir = os.path.join(base,'models')
log_dir = os.path.join(base,'logs')
model_file = model_dir + "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"

checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_acc', 
    save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True)

tensorboard = TensorBoard(
    log_dir=log_dir,
    batch_size=training_batch_size,
    update_freq = 'batch')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=5,
    cooldown=2,
    min_lr=1e-10,
    verbose=1)

#callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
callbacks = [checkpoint, reduce_lr, tensorboard]

InceptionV3 has bee used with non-trainable weights
Moreover a extra layers are added in the following manner:
Dense(512)->relu,
Dropout(0.15),
Dense(256)->relu,
Flatten(),
Dense(2)-> sigmoid (since 2 classes)

In [0]:
def create_model():
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape = target_size+(3,))
    x = inception_model.output
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Flatten()(x)
    predictions = Dense(2,activation='softmax')(x)
    model = Model(inputs = inception_model.input, outputs=predictions)
    for layer in inception_model.layers:
        layer.trainable = False
    return model

model is compiled, SGD optimizer is used

In [0]:
steps_per_epoch = len(train_data_generator)
validation_steps = len(val_data_generator)
classes = train_data_generator.classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(classes), classes)

optimizer = SGD()#Adam()
model = create_model()
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit_generator(train_data_generator,
                             steps_per_epoch=steps_per_epoch,
                             epochs=20,
                             verbose=1,
                             validation_data=val_data_generator,
                             validation_steps=validation_steps,
                             class_weight=class_weights,
                             callbacks=callbacks,
                             workers=16)

Epoch 1/20
33/33 [==============================] - 76s 2s/step - loss: 0.2364 - acc: 0.9021 - val_loss: 1.6341 - val_acc: 0.5625
Epoch 2/20
33/33 [==============================] - 77s 2s/step - loss: 0.2419 - acc: 0.9034 - val_loss: 2.0252 - val_acc: 0.5625
Epoch 3/20
33/33 [==============================] - 78s 2s/step - loss: 0.2371 - acc: 0.9021 - val_loss: 1.6619 - val_acc: 0.6250
Epoch 4/20
33/33 [==============================] - 77s 2s/step - loss: 0.2445 - acc: 0.9019 - val_loss: 1.6355 - val_acc: 0.5000
Epoch 5/20
33/33 [==============================] - 77s 2s/step - loss: 0.2418 - acc: 0.9021 - val_loss: 1.4047 - val_acc: 0.5000
Epoch 6/20
33/33 [==============================] - 78s 2s/step - loss: 0.2498 - acc: 0.8998 - val_loss: 2.8264 - val_acc: 0.6250
Epoch 7/20
33/33 [==============================] - 78s 2s/step - loss: 0.2323 - acc: 0.9063 - val_loss: 1.6799 - val_acc: 0.6250
Epoch 8/20
33/33 [==============================] - 78s 2s/step - loss: 0.2400 - acc: 0.90

Model has been run for 20 epochs here. Although it was tried for 80 epochs, the val_loss did not show consistent reduction for some reason

In [0]:
model_file_name = 'pneumonia_trained.hd5'
model.save(os.path.join(model_dir,model_file_name))

In [27]:
loaded_model = tf.keras.models.load_model(os.path.join(model_dir,model_file_name))
result = loaded_model.evaluate_generator(test_data_generator, steps=len(test_data_generator),verbose=1)
print('Loss: ', result[0])
print('Accuracy: ', result[1])

1/1 [==============================] - 14s 14s/step - loss: 1.2819 - acc: 0.6843
Loss:  1.2818865776062012
Accuracy:  0.6842949


In [33]:
import random
num_batches = len(test_data_generator)
random_batch = random.randint(0,num_batches-1)
y_img_batch, y_true_batch = test_data_generator[random_batch]
y_pred_batch = loaded_model.predict(y_img_batch)
batch_size = len(y_true_batch)
print('batch Size: ', batch_size)
print('Accuracy: ', np.mean(y_pred_batch.argmax(axis=-1)==y_true_batch.argmax(axis=-1))*100)

batch Size:  624
Accuracy:  69.87179487179486
